In [2]:
import os
import sys
from collections import OrderedDict
sys.path.append("../..")
import spatialpy
spatialpy.__file__
import matplotlib.pyplot as plt
import numpy
%load_ext autoreload
%autoreload 2

In [3]:
class Teleport(spatialpy.BoundaryCondition):
    def expression(self):
        return """
        if(isnan(me->x[0]) || me->x[0] > system->xhi){
            me->x[0] = system->xlo + 0.55;
        }
        if(isnan(me->x[1]) || me->x[1] > system->yhi){
            me->x[1] = system->yhi;
        }
        me->x[2] = 0;
        me->rho = 250.0;
        """

In [4]:
class HardFloor(spatialpy.BoundaryCondition):
    def expression(self):
        return """
        if(me->x[1] < 0.4){
            me->x[1] = 0.5;
            me->v[1] = 0.0;
        }
        """

In [5]:
class HardWall(spatialpy.BoundaryCondition):
    def expression(self):
        return """
        if(me->x[0] < 1){
            me->x[0] = system->xlo + 0.5;
            me->v[1] = 0.0;
        }
        """

In [6]:
class weir(spatialpy.Model):
    def __init__(self, model_name="weir_aka_backward_facing_step"):
        spatialpy.Model.__init__(self, model_name)

        xmax = 50.1
        ymax = 10.1

        # Construct mesh for backward facing step
        self.mesh = spatialpy.Mesh(0, xlim=(0,xmax), ylim=(0,ymax), zlim=(0,0), gravity=[0,-1,0])
        for x in numpy.arange(start=0, stop=xmax, step=0.1):
            for y in numpy.arange(start=0, stop=ymax, step=0.1):
                # Top and bottom
                if y == 0 or y == 10:
                    self.mesh.add_point([x,y,0], type=1, mass=10.0, vol=1.0, nu=1.0, fixed=True)
                # Water
                elif y > 1.5 and y < 8 and x > 1.5 and x < 8:
                    self.mesh.add_point([x,y,0], type=2, mass=0.1, vol=0.0001, nu=1.0, fixed=False)
                # Step
                elif x >= 10 and x <= 15 and y > 0 and y < 5:
                    self.mesh.add_point([x,y,0], type=1, mass=10.0, vol=1.0, nu=1.0, fixed=True)
                # Walls 
                elif x == 0 or x == 50:
                    self.mesh.add_point([x,y,0], type=1, mass=10.0, vol=1.0, nu=1.0, fixed=True)


        # Teleport particles back to start to simulate flow
        self.add_boundary_condition(Teleport())
        self.add_boundary_condition(HardFloor())
        #self.add_boundary_condition(HardWall())
        # Velocity
        #self.add_boundary_condition(spatialpy.BoundaryCondition(
        #    type_id=2,
        #    property = 'v',
        #   value    = [0.005, 0.0, 0.0] 
        #))

        self.staticDomain = False

        # Define time parameters
        self.timestep_size = 1e-3  # timestep size
        #self.num_timesteps = 50000    # number of timesteps
        #self.num_timesteps = 200000    # number of timesteps
        #self.output_freq = 400      # frequency of outputting results
        self.num_timesteps = 500    # number of timesteps
        self.output_freq = 5      # frequency of outputting results
        

In [7]:
model = weir()

In [10]:
from spatialpy import Solver
sol = Solver(model, debug_level=0)
%time sol.compile()

CPU times: user 1.37 s, sys: 2.85 ms, total: 1.37 s
Wall time: 40.9 s


In [23]:
%time result = sol.run()

cmd = cd /tmp/spatialpy_result_fbc2ztbj;/tmp/spatialpy_build_1kab3kmj/ssa_sdpd
got here1-
got here2
ERROR: nan/inf detected!!!
id=3962
got here3
x[0]=5.500000e-01
x[1]=5.000000e-01
x[2]=0.000000e+00
v[0]=-nan
v[1]=-nan
v[2]=-nan
vt[0]=0.000000e+00
vt[1]=0.000000e+00
vt[2]=0.000000e+00
F[0]=-6.643147e+16
F[1]=1.689361e+13
F[2]=-5.444719e-87
Fbp[0]=-1.559870e+04
Fbp[1]=-3.936050e+01
Fbp[2]=0.000000e+00
old_x[0]=7.398641e+00
old_x[1]=1.357588e+00
old_x[2]=0.000000e+00
old_v[0]=4.527332e+06
old_v[1]=-7.455955e+07
old_v[2]=3.723763e-97
sys->current_step=53
sys->dt=1.000000e-03
got empty line, ending
got here4
CPU times: user 16.2 ms, sys: 14.8 ms, total: 30.9 ms
Wall time: 17.5 s


In [13]:
def mpl_plot_property(t_ndx=0):
        import matplotlib.pyplot as plt
        
        points, data = result.read_step(t_ndx)

        d = data["type"]

        plt.figure(figsize=(6.4, 4.8))
        plt.scatter(points[:,0],points[:,1], c=d)
        plt.axis('scaled')
        #plt.colorbar()
        plt.title("t={0}".format(t_ndx))
        plt.grid(linestyle='', linewidth=1)
        plt.plot()
        return

In [18]:
timespan = []
time_vals = []
for i,t in enumerate(result.get_timespan()):
    time_vals.append(t)
    timespan.append(i)
print(timespan)
print(time_vals)

NameError: name 'result' is not defined

In [15]:
for i in timespan:
    try:
        mpl_plot_property(i)
    except Exception as e:
        print("t={0} {1}".format(i,e))
